In [274]:
import requests
from bs4 import BeautifulSoup
import time

url = "https://www.apartments.com/near-me/condos-for-rent/"
header = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15","Accept-Language": "en-gb","Accept-Encoding":"br, gzip, deflate","Accept":"test/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Referer":"http://www.google.com/"}

res = requests.get(url, headers=header)
soup = BeautifulSoup(res.text, "html.parser")

pages = int(soup.find("span", attrs={"class":"pageRange"}).text.split()[-1])

with open("apartments_rents.csv", mode='w', encoding='utf-8') as fd:
    fd.write("state,beds,baths,apartment_space_sqft,rent_price\n") # header
    for page in range(1, pages+1): # pageination
        print(f"collecting data of page {page}")
        page_link = f"https://www.apartments.com/near-me/condos-for-rent/{page}/"
        res = requests.get(page_link, headers=header)
        soup = BeautifulSoup(res.text, "html.parser")
        if (page == 1) or (page == 2):
            properties_tag = soup.find_all("div", attrs={"class":"top-level-info"})
        else:
            properties_tag = soup.find_all("div", attrs={"class":"property-information-wrapper"})
            
        location_tag = soup.find_all("div", attrs={"class":"property-information"})
        
        time.sleep(5)


        for i in range(len(properties_tag)):
            try:
                features_tag = properties_tag[i].find(attrs={"class":"property-beds"}).text.strip().split(" ")       
                rent_price = properties_tag[i].find(attrs={"class":"property-pricing"}).text.strip("$")
                
            except AttributeError:
                features_tag = properties_tag[i].find(attrs={"class":"bed-range"}).text.strip().split(" ")
                rent_price = properties_tag[i].find(attrs={"class":"price-range"}).text.strip("$")
                #rent_price = rent_price.split()
            
            if len(features_tag) <= 4 or '-' in features_tag:
                continue
            
            else:
                if len(features_tag) >= 7:
                    beds = features_tag[0]
                    baths = features_tag[2]
                    space = features_tag[4:] 
                    space = " ".join(space).strip().split()
                    space = space[0]
                    
                elif len(features_tag) == 6:
                    beds = features_tag[0].strip(",")
                    baths = features_tag[1]
                    space = features_tag[3:]
                    space = " ".join(space).strip().split()
                    space = space[0]

                location_tag = soup.find_all("div", attrs={"class":"property-information"})
                location_tag = location_tag[i].text.split()
                state = location_tag[-2].strip(",")
                fd.write(f"{state},\"{beds}\",\"{baths}\",\"{space}\",\"{rent_price}\"\n")
                

                time.sleep(5)


collecting data of page 1
collecting data of page 2
collecting data of page 3
collecting data of page 4
collecting data of page 5
collecting data of page 6
collecting data of page 7
collecting data of page 8
collecting data of page 9
collecting data of page 10
collecting data of page 11
collecting data of page 12
collecting data of page 13
collecting data of page 14
collecting data of page 15
collecting data of page 16
collecting data of page 17
collecting data of page 18


In [80]:
import pandas as pd

df = pd.read_csv("apartments_rents.csv")
df

,state,beds,baths,apartment_space_sqft,rent_price
0,IL,3,3.5,"3,300","5,295"
1,IL,3,2.0,"1,850","2,500"
2,IL,3,2.0,"1,619","2,950"
3,IL,1,1.0,925,"2,645"
4,IL,1,1.0,950,"2,100"
...,...,...,...,...,...
513,IL,2,1.0,"1,000","1,400"
514,IL,2,1.0,900,"1,395"
515,IL,1,1.0,650,"1,550"
516,IL,1,1.0,550,"1,100"


In [327]:
def scraping_condos(dicto):
    import requests
    from bs4 import BeautifulSoup
    import time
    
    with open("apartments_rents2.csv", mode='w', encoding='utf-8') as fd:
        fd.write("state,beds,baths,apartment_space_sqft,rent_price\n")
    
    for key, value in dicto.items():
        
        url = f"https://www.apartments.com/condos/{key}-{value}/"
    
        header = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15","Accept-Language": "en-gb","Accept-Encoding":"br, gzip, deflate","Accept":"test/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Referer":"http://www.google.com/"}

        res = requests.get(url, headers=header)
        soup = BeautifulSoup(res.text, "html.parser")

        pages = int(soup.find("span", attrs={"class":"pageRange"}).text.split()[-1])

        with open("apartments_rents2.csv", mode='a', encoding='utf-8') as fd:
            
            for page in range(1, pages+1): # pageination
                print(f"collecting data of page {page}")
                page_link = f"https://www.apartments.com/condos/{key}-{value}/{page}/"
                res = requests.get(page_link, headers=header)
                soup = BeautifulSoup(res.text, "html.parser")
                properties_tag = soup.find_all("div", attrs={"class":"top-level-info"})
                
                if properties_tag == []:
                    properties_tag = soup.find_all("div", attrs={"class":"property-information-wrapper"})

                time.sleep(2)


                for i in range(len(properties_tag)):
                    try:
                        features_tag = properties_tag[i].find(attrs={"class":"property-beds"}).text.strip().split(" ")       
                        rent_price = properties_tag[i].find(attrs={"class":"property-pricing"}).text.strip("$")

                    except AttributeError:
                        features_tag = properties_tag[i].find(attrs={"class":"bed-range"}).text.strip().split(" ")
                        rent_price = properties_tag[i].find(attrs={"class":"price-range"}).text.strip("$")
                        #rent_price = rent_price.split()

                    if (len(features_tag) <= 4) or ('-' in features_tag):
                        continue

                    else:
                        if len(features_tag) >= 7:
                            beds = features_tag[0]
                            baths = features_tag[2]
                            space = features_tag[4:] 
                            space = " ".join(space).strip().split()
                            space = space[0]

                        elif len(features_tag) == 6:
                            beds = features_tag[0].strip(",")
                            baths = features_tag[1]
                            space = features_tag[3:]
                            space = " ".join(space).strip().split()
                            space = space[0]
                            
                        try:
                            location_tag = soup.find_all("div", attrs={"class":"property-information"})
                            location_tag = location_tag[i].text.split()
                            state = location_tag[-2].strip(",")
                        
                        except IndexError:
                            location_tag = soup.find_all("div", attrs={"class":"property-title"})
                            location_tag = location_tag[i]["title"].split()
                            if len(location_tag[-1].strip(",")) == 2:
                                 state = location_tag[-1].strip(",")
                            else: 
                                state = location_tag[-2].strip(",")
                                
                            
                    fd.write(f"{state},\"{beds}\",\"{baths}\",\"{space}\",\"{rent_price}\"\n")


                    time.sleep(2)



In [324]:
states = {"kentucky":"ky", "new-york":"ny", "texas":"tx", "washington":"wa", 
          "virginia":"va", "tennessee":"tn", "hawaii":"hi", "new-jersey":"nj"}
scraping_condos(states)

collecting data of page 1
collecting data of page 2
collecting data of page 3
collecting data of page 4
collecting data of page 5
collecting data of page 1
collecting data of page 2
collecting data of page 3
collecting data of page 4
collecting data of page 5
collecting data of page 6
collecting data of page 7
collecting data of page 8
collecting data of page 9
collecting data of page 10
collecting data of page 11
collecting data of page 12
collecting data of page 13
collecting data of page 14
collecting data of page 15
collecting data of page 16
collecting data of page 17
collecting data of page 18
collecting data of page 1
collecting data of page 2
collecting data of page 1
collecting data of page 2
collecting data of page 3
collecting data of page 4
collecting data of page 5
collecting data of page 6
collecting data of page 7
collecting data of page 8
collecting data of page 9
collecting data of page 10
collecting data of page 11
collecting data of page 12
collecting data of page 13

In [137]:
df2 = pd.read_csv("apartments_rents2.csv")
df2

,state,beds,baths,apartment_space_sqft,rent_price
0,KY,1,1.0,901,"1,950"
1,KY,3,1.0,"1,000","1,350"
2,KY,4,2.0,"1,754","1,850"
3,KY,2,1.0,800,"1,595"
4,KY,2,2.5,"1,650","2,300"
...,...,...,...,...,...
1746,NY,3,1.0,105,"1,275"
1747,NY,2,1.0,775,"5,935"
1748,NY,3,2.0,"1,000","7,100"
1749,NY,4,2.0,"1,100","4,429"


In [138]:
df1 = pd.read_csv("apartments_rents.csv")
df1

,state,beds,baths,apartment_space_sqft,rent_price
0,IL,3,3.5,"3,300","5,295"
1,IL,3,2.0,"1,850","2,500"
2,IL,3,2.0,"1,619","2,950"
3,IL,1,1.0,925,"2,645"
4,IL,1,1.0,950,"2,100"
...,...,...,...,...,...
513,IL,2,1.0,"1,000","1,400"
514,IL,2,1.0,900,"1,395"
515,IL,1,1.0,650,"1,550"
516,IL,1,1.0,550,"1,100"


In [139]:
condos_rents = pd.concat([df1, df2])
condos_rents

,state,beds,baths,apartment_space_sqft,rent_price
0,IL,3,3.5,"3,300","5,295"
1,IL,3,2.0,"1,850","2,500"
2,IL,3,2.0,"1,619","2,950"
3,IL,1,1.0,925,"2,645"
4,IL,1,1.0,950,"2,100"
...,...,...,...,...,...
1746,NY,3,1.0,105,"1,275"
1747,NY,2,1.0,775,"5,935"
1748,NY,3,2.0,"1,000","7,100"
1749,NY,4,2.0,"1,100","4,429"


In [ ]:
condos_rents.to_csv("condos_rents.csv", index=False)

In [353]:
df = pd.read_csv("condos_rents.csv")
df

,state,beds,baths,apartment_space_sqft,rent_price
0,IL,3,3.5,3300,5295
1,IL,3,2.0,1850,2500
2,IL,3,2.0,1619,2950
3,IL,1,1.0,925,2645
4,IL,1,1.0,950,2100
...,...,...,...,...,...
2264,NY,3,1.0,105,1275
2265,NY,2,1.0,775,5935
2266,NY,3,2.0,1000,7100
2267,NY,4,2.0,1100,4429


In [141]:
df = pd.read_csv("condos_rents.csv")
df.head(30)

,state,beds,baths,apartment_space_sqft,rent_price
0,IL,3,3.5,"3,300","5,295"
1,IL,3,2.0,"1,850","2,500"
2,IL,3,2.0,"1,619","2,950"
3,IL,1,1.0,925,"2,645"
4,IL,1,1.0,950,"2,100"
5,IL,3,1.0,"1,500","2,100"
6,IL,3,2.0,"1,450","2,395"
7,IL,2,2.0,"1,180","2,995"
8,IL,2,2.0,"1,300","2,900"
9,IL,2,1.5,"1,100","2,700"


## Understand Data

In [86]:
df.columns

Index(['state', 'beds', 'baths', 'apartment_space_sqft', 'rent_price'], dtype='object')

In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2269 entries, 0 to 2268
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   state                 2269 non-null   object 
 1   beds                  2269 non-null   object 
 2   baths                 2269 non-null   float64
 3   apartment_space_sqft  2269 non-null   object 
 4   rent_price            2269 non-null   object 
dtypes: float64(1), object(4)
memory usage: 88.8+ KB


## Assuming studio = 1 bed

In [53]:
def replacing_studio(arg):
    for i in range(len(arg)):
        if type(arg[i]) == str:
            arg[i]
    return arg[i]

In [144]:
df['beds'].replace({"Studio":1}, inplace = True)
df['beds'] = df['beds'].astype(int)

C:\Users\user\AppData\Local\Temp\ipykernel_34828\3583780449.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['beds'].replace({"Studio":1}, inplace = True)


In [92]:
df['beds'][df['beds'] == '3-Feb']

39    3-Feb
Name: beds, dtype: object

In [77]:
df.head(40)

,state,beds,baths,apartment_space_sqft,rent_price
0,IL,3,3.5,"3,300","5,295"
1,IL,3,2.0,"1,850","2,500"
2,IL,3,2.0,"1,619","2,950"
3,IL,1,1.0,925,"2,645"
4,IL,1,1.0,950,"2,100"
5,IL,3,1.0,"1,500","2,100"
6,IL,3,2.0,"1,450","2,395"
7,IL,2,2.0,"1,180","2,995"
8,IL,2,2.0,"1,300","2,900"
9,IL,2,1.5,"1,100","2,700"


In [143]:
df.drop(df['beds'][df['beds'] == '3-Feb'].index, inplace=True)

In [168]:
def numbers_adj(arg):
    result = ""
    for i in range(len(arg)):
        arg[i].split()
        if arg[i] != ",":
            result = result + arg[i]
    return result



In [161]:
df['apartment_space_sqft'] = df['apartment_space_sqft'].apply(numbers_adj)

In [162]:
df['apartment_space_sqft']

0       3300
1       1850
2       1619
3        925
4        950
        ... 
2264     105
2265     775
2266    1000
2267    1100
2268     513
Name: apartment_space_sqft, Length: 2268, dtype: object

In [165]:
df['apartment_space_sqft'] = df['apartment_space_sqft'].astype(int)
df['apartment_space_sqft'] 

0       3300
1       1850
2       1619
3        925
4        950
        ... 
2264     105
2265     775
2266    1000
2267    1100
2268     513
Name: apartment_space_sqft, Length: 2268, dtype: int64

In [170]:
df['rent_price'] = df['rent_price'].apply(numbers_adj)
df['rent_price']

0       5295
1       2500
2       2950
3       2645
4       2100
        ... 
2264    1275
2265    5935
2266    7100
2267    4429
2268    5500
Name: rent_price, Length: 2268, dtype: object

In [175]:
df.drop(df['rent_price'][df['rent_price'] == 'Call For Rent'].index, inplace=True)

In [178]:
df.drop(df['rent_price'][df['rent_price'] == 'Call for Rent'].index, inplace=True)

In [180]:
df['rent_price'] = df['rent_price'].astype(int)
df['rent_price']

0       5295
1       2500
2       2950
3       2645
4       2100
        ... 
2264    1275
2265    5935
2266    7100
2267    4429
2268    5500
Name: rent_price, Length: 2267, dtype: int64

In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2267 entries, 0 to 2268
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   state                 2267 non-null   object 
 1   beds                  2267 non-null   int64  
 2   baths                 2267 non-null   float64
 3   apartment_space_sqft  2267 non-null   int64  
 4   rent_price            2267 non-null   int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 106.3+ KB


In [182]:
df.describe()

,beds,baths,apartment_space_sqft,rent_price
count,2267.000000,2267.000000,2267.000000,2267.000000
mean,1.756506,1.334142,917.550948,2972.477283
std,0.901991,0.565790,692.159846,2210.852674
min,1.000000,1.000000,100.000000,650.000000
25%,1.000000,1.000000,600.000000,1850.000000
50%,2.000000,1.000000,800.000000,2400.000000
75%,2.000000,2.000000,1085.000000,3330.000000
max,8.000000,4.000000,20000.000000,35000.000000


In [352]:
# beds:
# most of values between mean=1.7 and std is not that small (0.9)
# outliers in upper range (upper range > 25->75% of data)
# left-skewed distrubition (median > mean)

In [197]:
# baths:
# most of values between mean= 1.3 and std is small (0.5)
# outliers in upper range (upper range > 25->75% of data)
# normal distibution (mean = median)

In [ ]:
# apartment_space_sqft:
# most of values between mean= 917.5 and std is big (692.1)
# outliers in upper range (upper range > 25->75% of data)
# right-skewed distrubition (mean > median)

In [201]:
df['state'].unique()

array(['IL', 'IN', 'WI', 'KY', 'TX', 'VA', 'TN', 'HI', 'WA', 'NJ', 'NY'],
      dtype=object)

In [202]:
df['state'].nunique()

11

In [203]:
df['state'].value_counts()

state
IL    514
WA    493
NJ    383
NY    290
TN    237
HI    177
KY    136
IN     15
VA     12
TX      9
WI      1
Name: count, dtype: int64

In [268]:
df.drop(df['state'][df['state'] == 'WI'].index, inplace=True)

In [269]:
df['state'].value_counts()

state
IL    514
WA    493
NJ    383
NY    290
TN    237
HI    177
KY    136
IN     15
VA     12
TX      9
Name: count, dtype: int64

In [347]:
df.isna().sum()

state                   0
beds                    0
baths                   0
apartment_space_sqft    0
rent_price              0
dtype: int64

# What is the average monthly rent for each state?

In [354]:
result = df.groupby("state")[['rent_price']].mean().sort_values(by='rent_price', ascending=False).rename(columns={'rent_price':"avg_rent_price"})
result

,avg_rent_price
state,
NY,6045.651724
HI,3259.203390
NJ,2990.443864
WA,2625.393509
IL,2330.675097
TN,2196.848101
VA,1947.916667
KY,1434.411765
IN,1290.000000


# What is the most expensive state?

In [273]:
result[result['rent_price'] == result['rent_price'].max()]

,rent_price
state,
NY,6045.651724


# What is the cheapest state?

In [274]:
result[result['rent_price'] == result['rent_price'].min()]

,rent_price
state,
TX,1269.888889


# What is the cheapest apartment per each state?

In [355]:
df.groupby("state")[['rent_price']].min().sort_values(by='rent_price').rename(columns={'rent_price':"cheapest_rent_price"})

,cheapest_rent_price
state,
HI,650
KY,695
TX,695
WA,695
IL,750
IN,800
TN,900
NJ,900
VA,1000


# What is the most expensive apartment per each state?

In [331]:
df.groupby("state")[['rent_price']].max().sort_values(by='rent_price', ascending=False)

,rent_price
state,
NY,35000
IL,30000
HI,27000
NJ,14490
WA,9450
KY,8500
TN,5995
VA,3495
IN,2500


# What is the space of the most expensive apartment per each state?

In [344]:
df.groupby("state")[["apartment_space_sqft", "rent_price"]].max().sort_values(by='rent_price', ascending=False)


,apartment_space_sqft,rent_price
state,,
NY,2736,35000
IL,20000,30000
HI,3000,27000
NJ,9999,14490
WA,4844,9450
KY,9211,8500
TN,3040,5995
VA,2111,3495
IN,2088,2500


# What is the space of cheapest apartment per each state?

In [346]:
df.groupby("state")[["apartment_space_sqft", "rent_price"]].min().sort_values(by='rent_price')


,apartment_space_sqft,rent_price
state,,
HI,130,650
KY,450,695
TX,572,695
WA,115,695
IL,105,750
IN,550,800
TN,328,900
NJ,100,900
VA,460,1000


# Is there a correlation between rent price and number of beds?

In [282]:
df[['rent_price', 'beds']].corr()

#There is a correlation.

,rent_price,beds
rent_price,1.000000,0.115016
beds,0.115016,1.000000


# Is there a correlation between rent price and number of baths?

In [283]:
df[['rent_price', 'baths']].corr()

#There is a correlation. 

,rent_price,baths
rent_price,1.000000,0.299543
baths,0.299543,1.000000


# Is there a correlation between rent price and apartments's space?

In [285]:
df[['rent_price', 'apartment_space_sqft']].corr()

#There is a correlation. 

,rent_price,apartment_space_sqft
rent_price,1.000000,0.325726
apartment_space_sqft,0.325726,1.000000
